In [1]:
import os
import glob

In [47]:
val_dir = "./data/hhz/val/images/"
val_imgs = glob.glob(val_dir + "*.jpg")
print(len(val_imgs))
val_imgs.sort()
val_imgs[:5]

615


['./data/hhz/val/images/0003.jpg',
 './data/hhz/val/images/0013.jpg',
 './data/hhz/val/images/0023.jpg',
 './data/hhz/val/images/0030.jpg',
 './data/hhz/val/images/0031.jpg']

In [42]:
labels_txt = "./data/hhz/train/label.txt"
with open(labels_txt) as f:
    lines = f.readlines()

labels_dict = {}
i = 0
while i < len(lines):
    img_name = lines[i].strip()[2:]
    coords = lines[i+1].strip().split(" ")
    x1, y1 = int(coords[0]), int(coords[1])
    x2, y2 = x1 + int(coords[2]), y1 + int(coords[3])
    p1 = [coords[4], coords[5]]
    p2 = [coords[7], coords[8]]
    p3 = [coords[10], coords[11]]
    labels_dict[img_name] = [str(x1), str(y1), str(x2), str(y2)] + p1 + p2 + p3
    i += 2

In [34]:
save_dir = "/media/yee/WorkSpace/mAP-master/input/ground-truth/"
for img_path in val_imgs:
    img_name = os.path.basename(img_path)
    label = labels_dict.get(img_name)
    if label:
        text = " ".join(["rect"] + label) + "\n"
        save_path = os.path.join(save_dir, img_name.replace("jpg", "txt"))
        with open (save_path, 'w') as f:
            f.write(text)

In [79]:
gts_dir = "/media/yee/WorkSpace/mAP-master/input/ground-truth/"
preds_dir = "/media/yee/WorkSpace/mAP-master/input/detection-results/"

gts_txt = glob.glob(gts_dir + "*.txt")
preds_txt = glob.glob(preds_dir + "*.txt")
gts_txt.sort()
preds_txt.sort()
assert len(gts_txt) == len(preds_txt)

In [80]:
preds, gts = [], []
for (gt_file, pred_file) in zip(gts_txt, preds_txt):
    if "ch" in os.path.basename(gt_file):
        continue
    gf = open(gt_file)
    pf = open(pred_file)
#     print(gf.read().strip().split(" ")[-6:])
    gt = gf.read().strip().split(" ")[-6:]
    gt = list(map(float, gt))
    pred = [float(x) for x in pf.read().strip().split(" ")[-6:]]
    gts.append(gt)
    preds.append(pred)

In [81]:
import torch
import torch.nn as nn

preds = torch.tensor(preds)
gts = torch.tensor(gts)

loss = nn.L1Loss()
output = loss(preds, gts)
output.item()

8.808320045471191

tensor(1.0581)